In [1]:
import numpy as np
import pandas as pd
from sklearn import datasets

In [2]:
class Iris:
    def __init__(self,input_size,hidden_size,output_size,learning_rate):
        self.input_size=input_size
        self.output_size=output_size
        self.hidden_size=hidden_size
        self.learning_rate=learning_rate
        #Initialize weights and bias
        self.Vij=np.random.uniform(-0.5,0.5,(self.input_size,self.hidden_size))
        self.Wjk=np.random.uniform(-0.5,0.5,(self.hidden_size,self.output_size))
        self.Vbj=np.random.uniform(-0.5,0.5,(1,self.hidden_size))
        self.Wbk=np.random.uniform(-0.5,0.5,(1,self.output_size))

    
    def sigmoid(self,x):
        return 1/(1+np.exp(-x))
    def sigmoid_derivative(self,x):
        return x*(1-x)
    def forward_propogation(self,x):
            self.hidden_input=np.dot(x,self.Vij)+self.Vbj
            self.hidden_output=self.sigmoid(self.hidden_input)
            self.output_input=np.dot(self.hidden_output,self.Wjk) + self.Wbk
            self.output_final=self.sigmoid(self.output_input)
            return self.output_final
    def backword_propogation(self,x,y,output):
        error=y-output
        #Weight correction in weights
        delta_K=error * self.sigmoid_derivative(output)#Output layer error
        delta_in=delta_K.dot(self.Wjk.T)#Hidden Unit error
        delta_J=delta_in * self.sigmoid_derivative(self.hidden_output)#Hidden Layer error


        # Weigth adjustent in  Weight vector between Output and hidden layer
        self.Wjk+=self.hidden_output.T.dot(delta_K) * self.learning_rate

        #Change in bias between output and hidden layer
        self.Wbk+=np.sum(delta_K)*self.learning_rate
        
        # Weigth adjustent in  Weight vector between input and hidden layer
        self.Vij+=x.T.dot(delta_J) * self.learning_rate

        #Change in bias between input and hidden layer 
        self.Vbj+=np.sum(delta_J)*self.learning_rate
    def train_model(self,x,y,epochs):
        for epoch in range(epochs):
            output=self.forward_propogation((x))
            self.backword_propogation(x,y,output)
            if epoch % 1000 ==0 :
                print(f'Epoch{epoch}:Error{np.mean(np.abs(y-output))}')
    

In [8]:
data=datasets.load_iris()
df1= pd.DataFrame(data.data, columns=data.feature_names)
df1.head()
df2 = pd.DataFrame(data.target, columns=[data.target_names])
df2.head()

ValueError: Shape of passed values is (150, 1), indices imply (150, 3)

In [29]:
df['species'] = df['species'].replace({'Iris-setosa': 1, 'Iris-virginica': 0, 'Iris-versicolor': 0})
df.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,1
1,4.9,3.0,1.4,0.2,1
2,4.7,3.2,1.3,0.2,1
3,4.6,3.1,1.5,0.2,1
4,5.0,3.6,1.4,0.2,1


In [30]:
X = np.array([df['sepal_length'], df['sepal_width'], df['petal_length'], df['petal_width']]).T
Y = np.array(df['species'])

In [31]:
# Split data into training and testing sets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size=0.2, random_state=30)

# One-hot encode target labels for training
encoder = OneHotEncoder(categories='auto')
y_train_onehot = encoder.fit_transform(y_train.reshape(-1, 1)).toarray()

In [32]:
input_size = X_train.shape[1]
hidden_size = 8
output_size = y_train_onehot.shape[1]
learning_rate = 0.1
epochs = 10000

In [33]:
# Training
obj_ANN = Iris(input_size, hidden_size, output_size, learning_rate)
obj_ANN.train_model(X_train, y_train_onehot, epochs)

Epoch0:Error0.5378261250578386


Epoch1000:Error0.23190256064966283
Epoch2000:Error0.22954849644253525
Epoch3000:Error0.2292588923534707
Epoch4000:Error0.22962003343145462
Epoch5000:Error0.2302605311741501
Epoch6000:Error0.2294295772560626
Epoch7000:Error0.22910192341510568
Epoch8000:Error0.2289379240037134
Epoch9000:Error0.2284718726469615


In [38]:
output = obj_ANN.forward_propogation(X_test)
predicted_labels = np.argmax(output, axis=1)
accuracy = np.mean(predicted_labels == y_test.flatten())
print("Accuracy:", accuracy)



Accuracy: 0.03333333333333333
'Iris-Versicolor
'Iris-Versicolor
'Iris-Versicolor
Iris-setosa
Iris -Virginica
Iris -Virginica
Iris-setosa
Iris-setosa
Iris -Virginica
Iris-setosa
